In [83]:
import schedule_Generator as sg
import pandas as pd

Example
- Line 25: 25
- Stop ULB: 5407F
- Direction BOONDAEL GARE
- Date: SEP 07
- Time: 5-6 and after 20

In [84]:
# Input variables
route_id = 25
trip_headsign = 'BOONDAEL GARE'
date = 20210907
stop = '5407F'
nbusy_time = [['05:00:00', '07:00:00'], ['07:00:00', '09:00:00'], ['16:00:00', '20:00:00'], ['20:00:00', '25:00:00']]

In [85]:
stop_no_letter = ""
if stop[-1].isalpha():
    stop_no_letter = stop[:-1]
date_dt = "{}-{}-{}".format(str(date)[:4], str(date)[4:6], str(date)[-2:])

In [94]:
# get route short name
routes = pd.concat([pd.read_csv('../data/gtfs3Sept/routes.csv'), pd.read_csv('../data/gtfs23Sept/routes.csv')])
route_short_name = routes.loc[routes["route_id"]==route_id,:]["route_short_name"].values[0]

In [87]:
# Get day of week
day_of_week = pd.to_datetime(str(date), format='%Y%m%d').dayofweek
if day_of_week == 0:
    day_of_week = 'monday'
elif day_of_week == 1:
    day_of_week = 'tuesday'
elif day_of_week == 2:
    day_of_week = 'wednesday'
elif day_of_week == 3:
    day_of_week = 'thursday'
elif day_of_week == 4:
    day_of_week = 'friday'
elif day_of_week == 5:
    day_of_week = 'saturday'
elif day_of_week == 6:
    day_of_week = 'sunday'
day_of_week

'tuesday'

In [88]:
# combine time periods
new_nbusy_time = []
start = nbusy_time[0][0]
end = nbusy_time[0][1]
for i in range(1, len(nbusy_time)):
    if nbusy_time[i][0] == end:
        end = nbusy_time[i][1]
    else:
        new_nbusy_time.append([start, end])
        start = nbusy_time[i][0]
        end = nbusy_time[i][1]
new_nbusy_time.append([start, end])
new_nbusy_time

[['05:00:00', '09:00:00'], ['16:00:00', '25:00:00']]

In [89]:
# create datetime like list for actural time
new_nbusy_time_dt = []
for i in range(len(new_nbusy_time)):
    new_nbusy_time_dt.append([])
    for j in range(2):
        if new_nbusy_time[i][j] <= '24:00:00':
            new_nbusy_time_dt[i].append(date_dt+" "+new_nbusy_time[i][j])
        else:
            h = int(new_nbusy_time[i][j][:2]) - 24
            h = str(h)
            if len(h) == 1:
                h = "0"+h
            new_nbusy_time_dt[i].append((pd.to_datetime(date_dt, format='%Y-%m-%d')+pd.Timedelta(days=1)).strftime("%Y-%m-%d")+" "+(h+new_nbusy_time[i][j][2:]))
new_nbusy_time_dt

[['2021-09-07 05:00:00', '2021-09-07 09:00:00'],
 ['2021-09-07 16:00:00', '2021-09-08 01:00:00']]

In [102]:
trips = pd.concat([pd.read_csv('../data/gtfs3Sept/trips.csv'), pd.read_csv('../data/gtfs23Sept/trips.csv')])
trips_line = trips.loc[trips['route_id']==route_id,:]
trip_line_head = trips_line.loc[trips_line['trip_headsign']==trip_headsign,:]
trip_line_head.head()

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
96068,25,236896051,113149920236896051,BOONDAEL GARE,1,8925469,025t0256
96069,25,236896051,113149922236896051,BOONDAEL GARE,1,8925474,025t0256
96072,25,236896051,113149925236896051,BOONDAEL GARE,1,8925469,025t0256
96075,25,236896051,113149929236896051,BOONDAEL GARE,1,8925473,025t0256
96077,25,236896051,113149932236896051,BOONDAEL GARE,1,8925474,025t0256


In [103]:
calendar = pd.concat([pd.read_csv('../data/gtfs3Sept/calendar.csv'), pd.read_csv('../data/gtfs23Sept/calendar.csv')])
calendar_week = calendar.loc[((calendar['start_date']<=date) & (calendar['end_date']>=date)),:]
calendar_week_day = calendar_week.loc[calendar_week[day_of_week]==1,:]
calendar_week_day.head()

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
3,236270000,1,1,1,1,1,0,0,20210901,20210917
15,237645001,1,1,1,1,1,0,0,20210906,20210917
26,237648001,1,1,1,1,1,0,0,20210906,20210917
34,237534000,1,1,1,1,1,0,0,20210901,20210917
43,236695000,1,1,1,1,1,0,0,20210901,20210917


In [104]:
trip_line_date_head = pd.merge(left=trip_line_head, right=calendar_week_day, on='service_id', how='inner').loc[:,['route_id','service_id','trip_id']]
trip_line_date_head.head()

,route_id,service_id,trip_id
0,25,238202001,113480343238202001
1,25,238202001,113480346238202001
2,25,238202001,113480348238202001
3,25,238202001,113480356238202001
4,25,238202001,113480357238202001


In [108]:
stop_times = pd.concat([pd.read_csv('../data/gtfs3Sept/stop_times.csv'), pd.read_csv('../data/gtfs23Sept/stop_times.csv')])
time_line_date_head = pd.merge(left=trip_line_date_head, right=stop_times, on='trip_id')
time_line_date_head_stop = time_line_date_head.loc[time_line_date_head['stop_id']==stop,:]
time_line_date_head_stop.sort_values('arrival_time')
select = (time_line_date_head_stop['arrival_time']>=new_nbusy_time[0][0]) & (time_line_date_head_stop['arrival_time']<=new_nbusy_time[0][1])
for i in range(len(new_nbusy_time)):
    select = select | ((time_line_date_head_stop['arrival_time']>=new_nbusy_time[i][0]) & (time_line_date_head_stop['arrival_time']<=new_nbusy_time[i][1]))
time_line_date_head_stop_nbusy = time_line_date_head_stop.loc[select,:]
# time_line_date_head_stop_nbusy.sort_values('arrival_time')

/var/folders/v1/m2nqhx3d4yv9mjbpndrlsck00000gn/T/ipykernel_44170/2395021866.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  stop_times = pd.concat([pd.read_csv('../data/gtfs3Sept/stop_times.csv'), pd.read_csv('../data/gtfs23Sept/stop_times.csv')])


In [52]:
def unix_to_datetime(unix_time):
    return pd.to_datetime(unix_time,unit="ms",origin="unix")

def datetime_to_unix(dt):
    return int(pd.Timestamp.timestamp(pd.Timestamp(dt))*1000)

unix_to_datetime(1630998894287)

Timestamp('2021-09-07 07:14:54.287000')

In [100]:
actural_time = pd.concat([pd.read_csv('../data/vehiclePosition01.csv'),pd.read_csv('../data/vehiclePosition02.csv'),pd.read_csv('../data/vehiclePosition03.csv'),pd.read_csv('../data/vehiclePosition04.csv'),pd.read_csv('../data/vehiclePosition05.csv'),pd.read_csv('../data/vehiclePosition06.csv'),pd.read_csv('../data/vehiclePosition07.csv'),pd.read_csv('../data/vehiclePosition08.csv'),pd.read_csv('../data/vehiclePosition09.csv'),pd.read_csv('../data/vehiclePosition10.csv'),pd.read_csv('../data/vehiclePosition11.csv'),pd.read_csv('../data/vehiclePosition12.csv'),pd.read_csv('../data/vehiclePosition13.csv')])
actural_time_line = actural_time.loc[actural_time['LineID']==int(route_short_name),:]
actural_time_line_point = actural_time_line.loc[actural_time_line['PointID']==int(stop_no_letter),:]
actural_time_line_point['Time'] = actural_time_line_point['Time'].apply(unix_to_datetime)
actural_time_line_point_date = actural_time_line_point.loc[actural_time_line_point['Time'].dt.date == pd.to_datetime(date_dt).date(),:]
actural_time_line_point_date['Time'] = (actural_time_line_point_date['Time'] + pd.Timedelta('02:00:00'))
actural_time_line_point_date_arrive = actural_time_line_point_date.loc[actural_time_line_point_date['DistanceFromPoint']<=200,:]
select_list = [True]
for i in range(1, len(actural_time_line_point_date_arrive)):
    if (actural_time_line_point_date_arrive.iloc[i,0] - actural_time_line_point_date_arrive.iloc[i-1,0] <= pd.Timedelta('00:00:45')) and (actural_time_line_point_date_arrive.iloc[i,0] - actural_time_line_point_date_arrive.iloc[i-1,0] >= pd.Timedelta('00:00:15')):
        select_list.append(False)
    else:
        select_list.append(True)
actural_time_line_point_date_arrive_noduplicate = actural_time_line_point_date_arrive.loc[select_list,:]
select_list = [True]
for i in range(1, len(actural_time_line_point_date_arrive_noduplicate)):
    if (actural_time_line_point_date_arrive_noduplicate.iloc[i,0] - actural_time_line_point_date_arrive_noduplicate.iloc[i-1,0] <= pd.Timedelta('00:00:45')) and (actural_time_line_point_date_arrive_noduplicate.iloc[i,0] - actural_time_line_point_date_arrive_noduplicate.iloc[i-1,0] >= pd.Timedelta('00:00:15')):
        select_list.append(False)
    else:
        select_list.append(True)
actural_time_line_point_date_arrive_noduplicate = actural_time_line_point_date_arrive_noduplicate.loc[select_list,:]
select = (actural_time_line_point_date_arrive_noduplicate['Time']>=pd.to_datetime(new_nbusy_time_dt[0][0])) & (actural_time_line_point_date_arrive_noduplicate['Time']<=pd.to_datetime(new_nbusy_time_dt[0][1]))
for i in range(len(new_nbusy_time_dt)):
    select = select | ((actural_time_line_point_date_arrive_noduplicate['Time']>=pd.to_datetime(new_nbusy_time_dt[i][0])) & (actural_time_line_point_date_arrive_noduplicate['Time']<=pd.to_datetime(new_nbusy_time_dt[i][1])))
actural_time_line_point_date_arrive_noduplicate_nbusy = actural_time_line_point_date_arrive_noduplicate.loc[select,:]
actural_time_line_point_date_arrive_noduplicate_nbusy.sort_values('Time').count()

/var/folders/v1/m2nqhx3d4yv9mjbpndrlsck00000gn/T/ipykernel_44170/95322460.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actural_time_line_point['Time'] = actural_time_line_point['Time'].apply(unix_to_datetime)
/var/folders/v1/m2nqhx3d4yv9mjbpndrlsck00000gn/T/ipykernel_44170/95322460.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actural_time_line_point_date['Time'] = (actural_time_line_point_date['Time'] + pd.Timedelta('02:00:00'))


Time                 72
LineID               72
DirectionID          72
DistanceFromPoint    72
PointID              72
dtype: int64

In [101]:
on_time = 0
time_line_date_head_stop_nbusy['arrival_time'].apply(pd.to_timedelta)
actural_time_line_point_date_arrive_noduplicate_nbusy['Time'] - pd.Timestamp(date_dt+' 00:00:00')
for t in time_line_date_head_stop_nbusy['arrival_time'].apply(pd.to_timedelta):
    ot = False
    for at in actural_time_line_point_date_arrive_noduplicate_nbusy['Time'] - pd.Timestamp(date_dt+' 00:00:00'):
        t_dif = t - at
        if t_dif <= pd.Timedelta('00:00:45') and t_dif >= pd.Timedelta('-00:00:75'):
            ot = True
    if ot == True:
        on_time+=1
on_time_rate = on_time/len(time_line_date_head_stop_nbusy['arrival_time'])
on_time_rate

0.32222222222222224

In [18]:
actural_time_line_point_date_arrive_noduplicate.sort_values('Time')

,Time,LineID,DirectionID,DistanceFromPoint,PointID
1066927,2021-09-07 05:42:30.614,8,5229,6,1047
1087642,2021-09-07 06:13:40.483,8,5229,3,1047
1103465,2021-09-07 06:30:23.614,8,5229,0,1047
1121216,2021-09-07 06:46:04.691,8,5229,0,1047
1144937,2021-09-07 07:03:30.114,8,5229,16,1047
1162015,2021-09-07 07:14:48.016,8,5229,42,1047
1167906,2021-09-07 07:18:30.535,8,5229,0,1047
1180019,2021-09-07 07:26:03.562,8,5229,0,1047
1188912,2021-09-07 07:31:28.177,8,5229,0,1047
1198910,2021-09-07 07:37:27.300,8,5229,0,1047
